# Voting algorithm PoC

In [1]:
import random
from hashlib import sha256
from ecdsa import SigningKey

In [2]:
class Wallet:
    def __init__(self, name, balance):
        self.name = name
        self.balance = balance

    def __str__(self):
        return f"{self.name} has {self.balance}"

    def send(self, to, amount):
        self.balance -= amount
        to.balance += amount

    def receive(self, from_, amount):
        self.balance += amount
        from_.balance -= amount

In [3]:
class Candidate(Wallet):
    def __init__(self, name, balance, votes=0):
        super().__init__(name, balance)
        self.votes = votes

    def __str__(self):
        return f"{self.name} has {self.votes} votes"

    def vote(self, to):
        self.votes += 1
        to.votes -= 1
    
    def get_id(self):
        return sha256(self.name.encode()).hexdigest()

In [4]:
class Voter(Wallet):
    def __init__(self, name, balance, votes=1, voted=False):
        super().__init__(name, balance)
        self.votes = votes
        self._private_key = SigningKey.generate()

    def __str__(self):
        return f"{self.name} has {self.balance} and {self.votes} votes"

    def vote(self, to):
        self.votes -= 1
        to.votes += 1
    
    def get_id(self):
        return sha256(self.name.encode()).hexdigest()

    def sign_vote(self, who_I_vote):
        self.vote(who_I_vote)
        self.who_I_voted = who_I_vote.get_id().encode()
        return self._private_key.sign(self.who_I_voted)

    def validate_vote(self, vote):
        try:
            self._private_key.verifying_key.verify(vote, self.who_I_voted)
            return True
        except Exception as e:
            return False

In [5]:
class BlockVote():
    def __init__(self, voter, token, who_I_vote, weight):
        self.Voter = voter
        self.Token = token
        self.Who_I_vote = who_I_vote
        self.Weight = weight

In [6]:
class VotingAlgorithm:
    def __init__(self, candidates, voters, chain):
        self.candidates = candidates
        self.voters = voters
        self.chain = chain

    def __str__(self):
        return f"Candidates: {self.candidates} and Voters: {self.voters}"

    def vote(self):
        for voter in self.voters:
            who_I_vote = random.choice(self.candidates)
            # Signature
            bv = BlockVote(voter.get_id(), 1, voter.sign_vote(who_I_vote), 1)
            self.chain.append(bv)

    def show_results(self):
        for candidate in self.candidates:
            print(candidate)

In [7]:
def get_candidates():
    return [
        Candidate("Alice", 0),
        Candidate("Bob", 0),
        Candidate("Carol", 0),
        Candidate("Dave", 0),
    ]

In [8]:
candidates = get_candidates()
# Create the voters
voters = [
    Voter("Eve", 1),
    Voter("Frank", 1),
    Voter("Grace", 1),
    Voter("Hank", 1),
    Voter("Iris", 1),
    Voter("John", 1),
    Voter("Kathy", 1),
    Voter("Luke", 1),
    Voter("Mallory", 1),
    Voter("Nate", 1),
]

chain = []
# Create the voting algorithm
voting_algorithm = VotingAlgorithm(candidates, voters, chain)
# Run the voting algorithm
voting_algorithm.vote()

In [9]:
voting_algorithm.show_results()

Alice has 3 votes
Bob has 3 votes
Carol has 3 votes
Dave has 1 votes


# Digital signature of the votation

In [11]:
for voter in voters:
    print(voter.validate_vote(chain[0].Who_I_vote))

True
False
False
False
False
False
False
False
False
False
